# Taller de Física Computacional

Carlos Ruestes / Cristián Sánchez - Taller de Física Computacional - FCEN - UNCUYO

# Sesión 13: Tensión en una cadena de Lennard-Jones

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math as m
import scipy.optimize as opt

Como un caso particular de un problema de optimización en este *notebook* encontramos la configuración de mínima energía para una cadena de "atomos" que interaccionan a través del potencial de [Lennard-Jones](https://en.wikipedia.org/wiki/Lennard-Jones_potential). 

El potencial de Lennard Jones entre dos átomos separados por una distancia $r$ tiene la forma

$$ V_{\mathrm{lj}}(r) = 4\epsilon\left[ 
           \left( \frac{\sigma}{r} \right)^{12} 
           -  \left( \frac{\sigma}{r} \right)^{6} 
           \right] $$
           
donde los parámetros $\sigma$ y $\epsilon$ indican la profundidad del pozo de potencial y la distancia de equilibrio respectivamente.

In [ ]:
EPSILON = 10.0
SIGMA = 2.5

In [ ]:
def vlj(r):
    return EPSILON*((SIGMA / r)**12 - 2*(SIGMA / r)**6) 

In [ ]:
plt.plot(np.linspace(2.1,5,100),vlj(np.linspace(2.1,5,100)))

Como nos será necesaria definimos también la derivada del potencial respecto a la distancia interatómica

In [ ]:
def d_vlj(r):
    return EPSILON*( 12*SIGMA**6*r**(-7) -12*SIGMA**12*r**(-13) )

Comenzamos generando una estructura inicial para una cadena unidimensional de `NPART` partículas

In [ ]:
NPART = 25
D_EQ = SIGMA

In [ ]:
xcoords = np.linspace(0.0,D_EQ*(NPART-1),num=NPART)

In [ ]:
xcoords

Definimos la distancia en la recta entre un par de posiciones como el valor absoluto de la resta:

In [ ]:
def rij(xi,xj):
    return abs(xj - xi)

La función `d_rij` calcula la derivada de la distancia respecto del primer o segundo parámetro:

In [ ]:
def d_rij(xi,xj,k):
    if (xi - xj) > 0.0 and k == 1:
        return 1.0
    elif (xi - xj) < 0.0 and k == 1:
        return -1.0
    elif (xi - xj) > 0.0 and k == 2:
        return -1.0
    elif (xi - xj) < 0.0 and k == 2:
        return  1.0
    else:
        raise ValueError('k sólo puede ser 1 o 2.')

La energía potencial total de la cadena es la semisuma sobre todos los pares de la interacción entre pares

$$E(x_1,x_2,\ldots,x_{\mathrm{NPART}})) = \sum_{i,j=1,i\neq j}^{\mathrm{NPART}}V_{\mathrm{lj}}(r_{ij}) $$

La implementación que usamos no es la más eficiente ya que contiene un bulce sobre **todos** los pares de partículas diferentes.

In [ ]:
def energy(xs):
    energy = 0.0
    for i in range(0,NPART):
        for j in range(0,NPART):
            if (i != j):
                energy += vlj(rij(xs[i],xs[j]))
    return 0.5 * energy

- ¿Cómo podría hacerse más eficiente el cálculo de la energía?

La siguiente función calcula la derivada parcial de la energía respecto de cada coordenada de la forma:

$$\frac{\partial E}{\partial x_i} = \frac{d V}{d r_ij} \frac{\partial d(r_{ij})}{\partial x_i}$$ 

o 

$$\frac{\partial E}{\partial x_j} = \frac{d V}{d r_ij} \frac{\partial d(r_{ij})}{\partial x_j}$$ 

según corresponda y sumando. 

In [ ]:
def d_energy(xs,k):
    denergy = 0.0
    for i in range(0,NPART):
        for j in range(0,NPART):
            if (i != j):
                if k == i:
                    denergy += d_vlj(rij(xs[i],xs[j])) \
                    *d_rij(xs[i],xs[j],1)
                elif k == j:
                    denergy += d_vlj(rij(xs[i],xs[j])) \
                    *d_rij(xs[i],xs[j],2)   
    return denergy

Calcula el gradiente de la energía

$$ \nabla E = \left( \frac{\partial E}{\partial x_1}, \frac{\partial E}{\partial x_2}, \ldots,  \frac{\partial E}{\partial x_{\mathrm{NPART}}} \right)$$

In [ ]:
def grad_energy(xs):
    gradient = np.zeros((NPART))
    for i in range(0,NPART):
        gradient[i] = d_energy(xs,i)
    return gradient

Llama al optimizador con el método `BFGS` una tolerancia de $10^{-5}$ para minimizar la enrgía. Pasamos la función energía y su gradiente como parámetros y un punto inicial:

In [ ]:
res = opt.minimize(energy, xcoords, method='BFGS', tol=1e-5, jac=grad_energy)

El resultado se encuentra el vector `res.x`

In [ ]:
equilibrio = res.x

El largo de equilibrio es la distancia entre la última partícula y la primera. Notar que el largo es levemente menor que la distancia de equilibrio entre pares, ¿Porqué?

In [ ]:
largo = equilibrio[NPART-1] - equilibrio[0]

In [ ]:
print("El largo de la cadena es ",largo)

In [ ]:
y = np.zeros_like(equilibrio) + 0.0
plt.figure(figsize=(10.0,1.0))
plt.plot(equilibrio,y,marker = "o")

En el siguiente gráfico se muestra la magnitud de cada elemento del gradiente en la geometría de equilibrio:

In [ ]:
plt.plot(grad_energy(equilibrio))

Ahora complicamos un poco más las cosas, queremos encontrar el mínimo de energía cuando la cadena está sometida a una teensión de forma que su largo sea un valor predeterminado. Esto implica una optimización de la energía con la restricción de que la primera coodenada es cero y la última el `largo`.

In [ ]:
strains = np.linspace(0.95,1.1,50) # Este vector contiene las deformaciones
energies = np.zeros_like(strains)  # Este la energía para cada deformación
forcesa = np.zeros_like(strains)   # la fuerza sobre el primer elemento de la cadena
forcesb = np.zeros_like(strains)   # la fuerza sobre el último elemento
coords = np.zeros((NPART,strains.shape[0])) # las coordenadas que minimizan la energía

for i,strain in enumerate(strains):
    #las restricciones se pasan en esta estructura de datos en forma de funciones anónimas
    cons = ({'type': 'eq', 'fun': lambda x:  x[0] - 0.0},
        {'type': 'eq', 'fun': lambda x:  x[NPART - 1] - largo * strain}) # explicar la segunda restricción
    
    # para cada deformación llamamos el optimizador con un método que permite restricciones sobre las variables
    res = opt.minimize(energy, xcoords, method='SLSQP', tol=1e-5, jac=grad_energy, constraints=cons)
    
    #guardamos los resultados
    energies[i] = energy(res.x) #la energía total
    grad = grad_energy(res.x)  
    forcesa[i] = - grad[0] # la fuerza ejercida sobre el primer elemento
    forcesb[i] = - grad[NPART -1] #la fuerza ejercida sobre el último elemento
    coords[:,i] = res.x # las coordenadas en el mínimo
    print(i,res.success) # el paso y si el algoritmo fue exitoso o no
    xcoords = res.x

Aquí se grafica la energía en función de la deformación. Explique.

In [ ]:
plt.plot(strains,energies,marker="o")

Aquí se grafica la fuerza en función de la deformación, cuál es el régimen elástico, cuál el plástico? Explique la forma de la curva.

In [ ]:
plt.plot(strains,forcesa,marker="o")
plt.plot(strains,forcesb,marker="o")

Aquí se grafican las gometrías de equilibrio para cada deformación, la deformación es más grande para las cadenas que están más arriba. Explique.

In [ ]:
plt.figure(figsize=(10.0,10.0))
for i,strain in enumerate(strains):
    y = np.zeros_like(coords[:,i]) + i
    plt.plot(coords[:,i],y,marker="o")

Para investigar:

- ¿Cómo cambian las cosas si la cadena tiene más o menos partículas? Explique los resultados.
- Determine el módulo de deformación de la cadena definido como $\frac{\mathrm{fuerza}}{\mathrm{deformación}}$ para la geometría de equilibrio.
- ¿Para qué rango de deformaciones se puede considerar que el sistema se comporta de forma elástica?
- ¿Cómo cambian los regímenes elástico, plástico y tensión de ruptura con los parámetros del potencial interatómico?
- Cree una nueva `notebook` reepitiendo los cálculos para un potencial armónico entre partículas.